# Training PISA

$^*$: 意指詳見reference

先安裝mmdetection

In [ ]:
# 前面要先灌mmdetection
!python -m pip install --upgrade pip
!pip install openmim
!mim install mmdet==2.24.1

clone 我們的 github repo

In [ ]:
!git clone https://github.com/ShuYuHuang/pisa-simple-example.git
%cd pisa-simple-example

沒資料的話請下載資料

In [ ]:
!mkdir -p data/coco
!curl -L "https://public.roboflow.com/ds/teSUuBdtOy?key=MSfxYe5Fz3" > data/coco/roboflow.zip
!unzip -o data/coco/roboflow.zip -d data/coco
!rm -rf data/coco/roboflow.zip

## 1.Configure file

### 1.1.命名風格

MMDetection對於Configure File的命名風格如下：
{model}\_{backbone}\_{neck}\_{schedule}\_{dataset}

- {model}： model 種類，例如：faster_rcnn, mask_rcnn 等。
- {backbone}：Backbone 種類，例如：r50 (ResNet-50), x101 (ResNeXt-101) 等。
- {neck}：Neck 種類，例如：fpn, pafpn, nasfpn, c4 等。
- {schedule}： 訓練方案，選項是 1x、 2x、 20e 等。
> 1x 和 2x 分別代表 12 epoch 和 24 epoch，20e 在級聯 (Cascaded) 模型中使用，表示 20 epoch。
- {dataset}：資料集，例如：coco、 cityscapes、 voc_0712、 wider_face 等。

例如pisa_faster_rcnn_x101_32x4d_fpn_1x_coco.py的意思就是:
- 使用Prime sample attention(PISA)$^*$方案的FasterRCNN$^*$物件偵測模型 
- Backbone用的是ResNext101$^*$，每個cell用的是32個group做concatenate，並且寬度為4
- Neck用的是Feature Pyramid Networks(FPN)$^*$，在YOLOv4也有用
- 訓練上這邊用的是 12 epoch
- 訓練在coco dataset$^*$上

### 1.2. Configure基本內容

作者為了可以提高代碼複用率，所以 config 支持繼承的操作，通過 \_base\_ 變量來實現，\_base\_ 是一個 list 類型變量，裡面存放的是要繼承配置文件的路徑，任何配置文件都能往上追朔繼承以下四種類型的文件：
- 模型 (models)
- 資料集 (datasets)
- 訓練策略 (schedules)
- 運行時的默認配置 (default_runtime)

在運算過程中mmdetection會把多個config檔整合成一個，所以後面我們下載下來的基本pisa config檔只有一個

這邊看一下pisa_faster_rcnn_x101_32x4d_fpn_1x_coco.py 長怎樣：

In [1]:
# 安裝mmdet時會順便安裝mmcv，是mm系列有關cv演算法的核心，其中包含config讀取
from mmcv import Config

In [2]:
# 沒有config檔的話請先下載
!mkdir downloads
!mim download mmdet --config pisa_faster_rcnn_x101_32x4d_fpn_1x_coco --dest ./downloads
cfg=Config.fromfile("downloads/pisa_faster_rcnn_x101_32x4d_fpn_1x_coco.py")

processing pisa_faster_rcnn_x101_32x4d_fpn_1x_coco...
pisa_faster_rcnn_x101_32x4d_fpn_1x_coco-e4accec4.pth exists in /home/jovyan/course/pisa-simple-example/downloads
Successfully dumped pisa_faster_rcnn_x101_32x4d_fpn_1x_coco.py to /home/jovyan/course/pisa-simple-example/downloads


大致上包含資料、模型、訓練、測試、推論的每個設定

In [3]:
cfg.keys()

dict_keys(['model', 'dataset_type', 'data_root', 'img_norm_cfg', 'train_pipeline', 'test_pipeline', 'data', 'evaluation', 'optimizer', 'optimizer_config', 'lr_config', 'runner', 'checkpoint_config', 'log_config', 'custom_hooks', 'dist_params', 'log_level', 'load_from', 'resume_from', 'workflow', 'opencv_num_threads', 'mp_start_method', 'auto_scale_lr'])

### 1.3.Model設定

Model裡面就把架構有系統的建構方式設定起來，也有有關訓練和測試的設定

In [21]:
cfg.model.keys()

dict_keys(['type', 'backbone', 'neck', 'rpn_head', 'roi_head', 'train_cfg', 'test_cfg'])

#### 1.3.1 類型


物件偵測模型類型，由於PISA演算法的核心是在roi_head的cnn blocks中實施self attention，所以與這個無關

但是在論文上主要有使用到FasterRCNN來做主架構更動

In [22]:
cfg.model.type

'FasterRCNN'

#### 1.3.2 Backbone模型

同個物件偵測模型中的backbone模型可以換置，會影響圖片的embedding效果，我們這邊用論文中做得較好的ResNext模型。

In [23]:
cfg.model.backbone

{'type': 'ResNeXt',
 'depth': 101,
 'num_stages': 4,
 'out_indices': (0, 1, 2, 3),
 'frozen_stages': 1,
 'norm_cfg': {'type': 'BN', 'requires_grad': True},
 'norm_eval': True,
 'style': 'pytorch',
 'init_cfg': {'type': 'Pretrained',
  'checkpoint': 'open-mmlab://resnext101_32x4d'},
 'groups': 32,
 'base_width': 4}

#### 1.3.3 Neck模型

Neck 模型來將對應不同scale投射到不同解析度輸出時使用，使用FPN$^*$

In [7]:
cfg.model.neck

{'type': 'FPN',
 'in_channels': [256, 512, 1024, 2048],
 'out_channels': 256,
 'num_outs': 5}

#### 1.3.4 Region Proposal Network Head

使用Region Proposal Network$^*$ 以前面Neck的資訊做物件區域偵測的提案。

In [8]:
cfg.model.rpn_head

{'type': 'RPNHead',
 'in_channels': 256,
 'feat_channels': 256,
 'anchor_generator': {'type': 'AnchorGenerator',
  'scales': [8],
  'ratios': [0.5, 1.0, 2.0],
  'strides': [4, 8, 16, 32, 64]},
 'bbox_coder': {'type': 'DeltaXYWHBBoxCoder',
  'target_means': [0.0, 0.0, 0.0, 0.0],
  'target_stds': [1.0, 1.0, 1.0, 1.0]},
 'loss_cls': {'type': 'CrossEntropyLoss',
  'use_sigmoid': True,
  'loss_weight': 1.0},
 'loss_bbox': {'type': 'L1Loss', 'loss_weight': 1.0}}

#### 1.3.5 !!!Region of Interest Head!!!

使用有計算proposal 對ground truth attention的Region of Interest$^*$ head 將最終預測算出。

*這邊就是PISA將Attention加進model的地方*

roi_head會做一些基本的
- bbox_roi_extractor: ROI pooling相關設定
- bbox_head: Bounding box prediction head的設定，output head channel數、roi feature的大小、class數等等

In [9]:
cfg.model.roi_head

{'type': 'PISARoIHead',
 'bbox_roi_extractor': {'type': 'SingleRoIExtractor',
  'roi_layer': {'type': 'RoIAlign', 'output_size': 7, 'sampling_ratio': 0},
  'out_channels': 256,
  'featmap_strides': [4, 8, 16, 32]},
 'bbox_head': {'type': 'Shared2FCBBoxHead',
  'in_channels': 256,
  'fc_out_channels': 1024,
  'roi_feat_size': 7,
  'num_classes': 80,
  'bbox_coder': {'type': 'DeltaXYWHBBoxCoder',
   'target_means': [0.0, 0.0, 0.0, 0.0],
   'target_stds': [0.1, 0.1, 0.2, 0.2]},
  'reg_class_agnostic': False,
  'loss_cls': {'type': 'CrossEntropyLoss',
   'use_sigmoid': False,
   'loss_weight': 1.0},
  'loss_bbox': {'type': 'SmoothL1Loss', 'loss_weight': 1.0, 'beta': 1.0}}}

其他PISA實現演算法相關參數設定在cfg.model.train_cfg.rcnn中

##### Importance-based Sample Reweighting(ISR)

都是要做Sample reweighting時做Hierarchical Local Rank(HLR)時用到的參數

在ISR中給negative proposal用Score做的HLR後計算attention式子的參數:
- k: 調整attention分布尖銳程度用，對高HLR的偏袒程度
- bias: 調整最低attention數值用

In [19]:
cfg.model.train_cfg.rcnn.sampler

{'type': 'ScoreHLRSampler',
 'num': 512,
 'pos_fraction': 0.25,
 'neg_pos_ub': -1,
 'add_gt_as_proposals': True,
 'k': 0.5,
 'bias': 0.0}

ISR中給positive proposal用IOU做的HLR計算attention式子的的參數:
- k: 調整attention分布尖銳程度用，對高HLR的偏袒程度
- bias: 調整最低attention數值用

In [17]:
cfg.model.train_cfg.rcnn.isr

{'k': 2, 'bias': 0}

##### Classification-Aware Regression Loss(CARL)

使用class置信度計算做底計算Regression時的attention算式用到的參數:
- k: 調整attention分布尖銳程度用，對高機率的偏袒程度
- bias: 調整最低attention數值用

In [18]:
cfg.model.train_cfg.rcnn.carl

{'k': 1, 'bias': 0.2}

### 1.4.資料設定

- xxx_per_gpu: 一些GPU設定
- data.train/val/test: 訓練、驗證、測試集都有不同的設定 (下面以train來講解)
- data.train.type: 資料集類型，主要是標註的格式
- data.train.ann_file: 標註檔位置
- data.train.img_prefix: 圖檔資料夾位置
- data.train.pipeline: 用list方式列出pre-process會用的演算法，可能有normalize大小、翻轉、旋轉等等可以用

In [4]:
print(cfg.data.train.type)
print(cfg.data.train.ann_file)
print(cfg.data.train.img_prefix)

CocoDataset
data/coco/annotations/instances_train2017.json
data/coco/train2017/


In [32]:
print(*cfg.data.train.pipeline,sep='\n')

{'type': 'LoadImageFromFile'}
{'type': 'LoadAnnotations', 'with_bbox': True}
{'type': 'Resize', 'img_scale': (1333, 800), 'keep_ratio': True}
{'type': 'RandomFlip', 'flip_ratio': 0.5}
{'type': 'Normalize', 'mean': [123.675, 116.28, 103.53], 'std': [58.395, 57.12, 57.375], 'to_rgb': True}
{'type': 'Pad', 'size_divisor': 32}
{'type': 'DefaultFormatBundle'}
{'type': 'Collect', 'keys': ['img', 'gt_bboxes', 'gt_labels']}


### 1.5. 更改cfg內容

藉由引用cfg檔我們可以繼承屬性，並只修改其中一部分，我們可以簡單做出一個符合資料集以及model設定的cfg檔

#### 1.5.1 自編cfg檔內容

In [5]:
# 想要看元檔可以跑這行
!cat pisa_chess.py

# The new config inherits a base config to highlight the necessary modification
_base_ = ['downloads/pisa_faster_rcnn_x101_32x4d_fpn_1x_coco.py']

# We also need to change the num_classes in head to match the dataset's annotation
model = dict(
    roi_head=dict(
        bbox_head=dict(
            num_classes=13)),
    train_cfg=dict(
        rpn_proposal=dict(
            nms_pre=200,
            max_per_img=200)),
    test_cfg=dict(
        rpn=dict(
            nms_pre=200,
            max_per_img=200))
)
# Modify dataset related settings
dataset_type = 'COCODataset'
classes = ('bishop',
'black-bishop',
 'black-king',
 'black-knight',
 'black-pawn',
 'black-queen',
 'black-rook',
 'white-bishop',
 'white-king',
 'white-knight',
 'white-pawn',
 'white-queen',
 'white-rook')
data_root = 'data/coco/'

data = dict(
    train=dict(
        img_prefix='data/coco/train/',
        classes=classes,
        ann_file='data/coco/train/_annotations.coco.json'),
    val=dict(
        img_prefix='

**繼承**

先繼承一個已經很完整的設定檔，其中已經有關於model, data, training, device相關設定
```python
_base_ = ['downloads/pisa_faster_rcnn_x101_32x4d_fpn_1x_coco.py']
```

**修改模型設定**

``` python
model = dict(
    roi_head=dict(
        bbox_head=dict(
            num_classes=13)),
    train_cfg=dict(
        rpn_proposal=dict(
            nms_pre=200,
            max_per_img=200)),
    test_cfg=dict(
        rpn=dict(
            nms_pre=200,
            max_per_img=200))
)
```
- roi_head的class數量:這最重要，因為基礎設定是coco的80類別，這邊就看我們的任務要幾類
- train_cfg.rpn_proposal: non-maximum suppression的數量設定，如果比coco簡單或許可以設小一點加快訓練
- test_cfg.rpn: non-maximum suppression的數量設定

**類別與母資料夾名稱**

這邊要將classes改成自己對應的label(照順序)，並且給定data存放的母資料夾
```python
classes = ('bishop',
'black-bishop',
 'black-king',
 'black-knight',
 'black-pawn',
 'black-queen',
 'black-rook',
 'white-bishop',
 'white-king',
 'white-knight',
 'white-pawn',
 'white-queen',
 'white-rook')
data_root = 'data/coco/'
```

**修改資料讀取設定**

指定資料和標註給訓練、驗證、測試使用
```python
data = dict(
    train=dict(
        img_prefix='data/coco/train/',
        classes=classes,
        ann_file='data/coco/train/_annotations.coco.json'),
    val=dict(
        img_prefix='data/coco/valid/',
        classes=classes,
        ann_file='data/coco/valid/_annotations.coco.json'),
    test=dict(
        img_prefix='data/coco/test/',
        classes=classes,
        ann_file='data/coco/test/_annotations.coco.json'))
```
- img_prefix: 影像資料夾名稱
- classes: 用剛剛指定的class名稱
- ann_file: 標註檔

想要修改augmentation可以修改pipeline，例如:
```python
    test=dict(
        img_prefix='data/coco/test/',
        classes=classes,
        ann_file='data/coco/test/_annotations.coco.json',
        pipeline=[
            dict(type='LoadImageFromFile'),
            dict(
                type='MultiScaleFlipAug',
                img_scale=(1333, 800),
                flip=False,
                transforms=[
                    dict(type='Resize', keep_ratio=True),
                    dict(type='RandomFlip'),
                    dict(
                        type='Normalize',
                        mean=[123.675, 116.28, 103.53],
                        std=[58.395, 57.12, 57.375],
                        to_rgb=True),
                    dict(type='Pad', size_divisor=32),
                    dict(type='ImageToTensor', keys=['img']),
                    dict(type='Collect', keys=['img'])
                ])
        ])
```

## 2. Training

有了設定檔就可以直接開始train

In [1]:
# 可以先看看用法指南，不過蠻多的也容易混亂，可以參考後面簡單的使用方式
!python tools/train.py --help

usage: train.py [-h] [--work-dir WORK_DIR] [--resume-from RESUME_FROM]
                [--auto-resume] [--no-validate]
                [--gpus GPUS | --gpu-ids GPU_IDS [GPU_IDS ...] | --gpu-id
                GPU_ID] [--seed SEED] [--diff-seed] [--deterministic]
                [--options OPTIONS [OPTIONS ...]]
                [--cfg-options CFG_OPTIONS [CFG_OPTIONS ...]]
                [--launcher {none,pytorch,slurm,mpi}]
                [--local_rank LOCAL_RANK] [--auto-scale-lr]
                config

Train a detector

positional arguments:
  config                train config file path

optional arguments:
  -h, --help            show this help message and exit
  --work-dir WORK_DIR   the dir to save logs and models
  --resume-from RESUME_FROM
                        the checkpoint file to resume from
  --auto-resume         resume from the latest checkpoint automatically
  --no-validate         whether not to evaluate the checkpoint during training
  --gpus GPUS           (Depr

In [ ]:
# 最簡單來個config檔就結束了
!python tools/train.py pisa_chess.py 

In [2]:
# 若想試試看不加PISA的結果也可以download相對版本:

# !mim download mmdet --config faster_rcnn_x101_32x4d_fpn_1x_coco --dest ./downloads
# !python tools/train.py fasterrcnn_chess.py

## Exercise

請練習自己基於pisa_chess.py改寫一個configure file, 將:
1. nms_pre數量調整為2000
2. 總 epoch數調整為20
3. optimizer的learning rate初始值調為0.001

hint:
新增一個script: e.g. answer_2.py，內容包含
```python
    _base_ = [??]
    model = dict(??)
    optimizer = ?? 0.001 ??
    ?? = ?? max_epochs=20 ??
```

## Reference

* [PISA] Cao, Y., Chen, K., Loy, C. C., & Lin, D. (2020). Prime sample attention in object detection. In Proceedings of the IEEE/CVF Conference on Computer Vision and Pattern Recognition (pp. 11583-11591).
* [ResNext] Xie, S., Girshick, R., Dollár, P., Tu, Z., & He, K. (2017). Aggregated residual transformations for deep neural networks. In Proceedings of the IEEE conference on computer vision and pattern recognition (pp. 1492-1500).
* [FPN] Lin, T. Y., Dollár, P., Girshick, R., He, K., Hariharan, B., & Belongie, S. (2017). Feature pyramid networks for object detection. In Proceedings of the IEEE conference on computer vision and pattern recognition (pp. 2117-2125).
* [COCO Dataset] Lin, T. Y., Maire, M., Belongie, S., Hays, J., Perona, P., Ramanan, D., ... & Zitnick, C. L. (2014, September). Microsoft coco: Common objects in context. In European conference on computer vision (pp. 740-755). Springer, Cham.